In [ ]:
import pandas as pd
import urllib3
import json
import os
import inflection

from bs4 import BeautifulSoup
from io import BytesIO
from zipfile import ZipFile

import sqlite3
from sqlite3 import Error

from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, Float

In [ ]:
#Com urllib3 temos que iniciar um PoolManager - aqui estou fazendo a request do site da câmara dos deputados

http = urllib3.PoolManager()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) #Mensagem chata desativada
response = http.request('GET', 'https://dadosabertos.camara.leg.br/swagger/api.html#staticfile')

In [ ]:
#Utilizando o BeautifulSoup para parsear o HTML

soup = BeautifulSoup(response.data, 'html.parser')

In [ ]:
#Definindo diretorios

BASE_DIR = os.path.abspath('')
DEPUTADOS_DIR = os.path.join( BASE_DIR, 'dados_deputados' )
DATA_DIR = os.path.join( BASE_DIR, 'json_files' )

try:
    os.mkdir( DEPUTADOS_DIR )
except:
    pass

try:
    os.mkdir( DATA_DIR )
except:
    pass

In [ ]:
#Definindo a funcao de extracao de json

def extracao_json_zip(url, destination_dir):
    
    if url.endswith('.json.zip'):
        
        zipped_json = http.request('GET', url, preload_content=False)
        zipfile = ZipFile(BytesIO(zipped_json.read()))
        list_of_files = zipfile.namelist()
        
        for file_name in list_of_files:
            if file_name.endswith('.json'):

                zipfile.extract('{}'.format(file_name), destination_dir)

#Extraindo os arquivos json dos zips

for link in soup.find_all('a'):
    url = link.get('href')
    extracao_json_zip(url, DATA_DIR)

In [ ]:
url_legislaturas = 'https://dadosabertos.camara.leg.br/api/v2/legislaturas'
response = http.request('GET', url_legislaturas)

legislaturas = json.loads(response.data)
ids_legislaturas = [leg['id'] for leg in legislaturas['dados']]

In [ ]:
#Extraindo dados de legislaturas

for leg in ids_legislaturas:
    
    url_deputados = f'https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura={leg}'

    response = http.request('GET', url_deputados)
    deputados = json.loads(response.data)
    deputados = pd.DataFrame(deputados['dados'])
    deputados.to_csv(os.path.join(DEPUTADOS_DIR,f'legislatura - {leg}.csv'), encoding='utf-8', index=False)

In [ ]:
#Codigo para pegar todas as colunas e os tipos de dados

columns = {}

for f in os.listdir(DATA_DIR):
    destination = os.path.join(DATA_DIR, f)
    expenses_by_year = json.load(open(destination))
    
    for item in expenses_by_year['dados']:
        for key, value in item.items():
            if key not in columns:
                columns[key] = set()
            columns[key].add(type(value))

In [ ]:
columns

In [ ]:
#Criando uma nova DB SQLite

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:                     
            conn.close()


if __name__ == '__main__':
    create_connection("camara_federal.db")

In [ ]:
#Criando uma engine SQLAlchemy

engine = create_engine('sqlite:///camara_federal.db', echo=True)
metadata = MetaData(bind=engine)

In [ ]:
table = Table('expenses', metadata,
  Column('ano', Integer()),
  Column('cnpj_cpf', String(255)),
  Column('codigo_legislatura', Integer()),
  Column('cpf', String(255)),
  Column('data_emissao', DateTime()),
  Column('descricao', String(255)),
  Column('descricao_especificacao', String(255)),
  Column('fornecedor', String(255)),
  Column('id_deputado', Integer()),
  Column('id_documento', Integer()),
  Column('legislatura', Integer()),
  Column('lote', String(255)),
  Column('mes', Integer()),
  Column('nome_parlamentar', String(255)),
  Column('numero', String(255)),
  Column('numero_carteira_parlamentar', String(255)),
  Column('numero_deputado_id', Integer()),
  Column('numero_especificacao_sub_cota', Integer()),
  Column('numero_sub_cota', Integer()),
  Column('parcela', Integer()),
  Column('passageiro', String(255)),
  Column('ressarcimento', String(255)),
  Column('restituicao', String(255)),
  Column('sigla_partido', String(255)),
  Column('sigla_uf', String(255)),
  Column('tipo_documento', String(255)),
  Column('trecho', String(255)),
  Column('url_documento', String(255)),
  Column('valor_documento', Float()),
  Column('valor_glosa', Float()),
  Column('valor_liquido', Float()),
)

In [ ]:
table = Table('legislatures', metadata,
  Column('email', String(255)),
  Column('id', Integer()),
  Column('id_legislatura', Integer()),
  Column('nome', String(255)),
  Column('sigla_partido', String(255)),
  Column('sigla_uf', String(255)),
  Column('uri', String(255)),
  Column('uri_partido', String(255)),
  Column('url_foto', String(255)),
)

In [ ]:
metadata.create_all()

In [ ]:
underscore_columns = [inflection.underscore(column) for column in columns]

for f in os.listdir(DATA_DIR):    

    with open(os.path.join(DATA_DIR, f), "r") as json_file:
        
        #Ler cada um dos jsons
        
        data = json.load(json_file)
        data = pd.DataFrame(data['dados'])
        
        #Transformar as colunas de cada arquivo em underscore
        
        data.columns = [inflection.underscore(column) for column in data.columns]
         
        
        data['valor_glosa'] = data['valor_glosa'].astype(float, errors='raise')
        data['valor_liquido'] = data['valor_liquido'].astype(float, errors='raise')
        data['valor_documento'] = data['valor_documento'].astype(float, errors='raise')
        data['data_emissao'] = pd.to_datetime(data['data_emissao'], errors='coerce')

        data.to_sql('expenses', engine, if_exists='append', index=False, chunksize=20000)

In [ ]:
#Load legislatures

for f in os.listdir( DEPUTADOS_DIR ):
    if f.endswith('.csv'):
        data = pd.read_csv(os.path.join(DEPUTADOS_DIR, f))
        data.columns = [inflection.underscore(column) for column in data.columns]
        
        #Upper para garantir que todos os nomes estejam em maiusculo
        data['nome'] = data['nome'].str.upper()
        data.to_sql('legislatures', engine, if_exists='append', index=False, chunksize=100)
    

In [ ]:
#Fechar a engine
engine.dispose()